## Importing Packages 

In [1]:
import cv2
import time
import numpy as np

## Creating output file

In [12]:
fourcc= cv2.VideoWriter_fourcc(*'DIVX')
out=cv2.VideoWriter('harry_potter.avi',fourcc,20.0, (640,480))

## Recording Background and Pass it over the Cloak Area

In [13]:
cap=cv2.VideoCapture(0)
time.sleep(5)                                                                  #Waiting for the webcam to open 
count=0
background=0

#Recording the background Area for the range of 60
for i in range(60):
    ret,background=cap.read()
background=np.flip(background, axis=1)                                         #Obtain the mirror image by flipping pixel values of the image


#Implementing Invisible Cloak by Passing the background portion over the Area covered by cloak for every frame.
while (cap.isOpened()):
    ret,img=cap.read()
    if not ret: #Case when webcam does not return any frame 
        break
    count+=1
    img=np.flip(img,axis=1)
    hsv=cv2.cvtColor(img, cv2.COLOR_BGR2HSV)                                   #Conversion of image type to HSV to detect the red color of our cloak
    
    #Lower range for Red Mask
    lower_red=np.array([0,120,70])
    upper_red=np.array([9,255,255])
    mask1=cv2.inRange(hsv,lower_red,upper_red)
    
    #Upper range for Red Mask
    lower_red=np.array([171,120,70])
    upper_red=np.array([180,255,255])
    mask2=cv2.inRange(hsv,lower_red,upper_red)
    
    #Final Mask Region
    mask1=mask1+mask2
    
    #Removing Noise and Smoothening the Masked Area
    mask1=cv2.morphologyEx(mask1, cv2.MORPH_OPEN,np.ones((3,3),np.uint8))      #Erosion Followed by Dilation 
    mask1=cv2.morphologyEx(mask1, cv2.MORPH_DILATE,np.ones((3,3),np.uint8))    #Dilation

    mask2=cv2.bitwise_not(mask1)                                               #Inverted mask of Mask 1
    res1=cv2.bitwise_and(img,img,mask=mask2)                                   #Applying the total frame over the masked region by bitwise_and so that unmasked area does not contain anything from the current frame.
    res2=cv2.bitwise_and(background,background,mask=mask1)                     #Similar to previous step but in this case we pass the background we recorded before in the masked area and unmasked area remains unaffected.
    
    finalOutput=cv2.addWeighted(res1,1,res2,1,0)                               #It blends both Image together, and since both res1 and res2 didn't have anything in unmasked region they add up together without affecting each other.
    out.write(finalOutput)                                                     #Writing final Output produced in the same directory
    cv2.imshow("magic",finalOutput) 
    if cv2.waitKey(1) & 0xFF == ord('q'):                                      #Wait for 1ms for user response and if he enters "q" in his/her keyboard, the loop is terminated and webcam window closes.
        break
    
cap.release()
out.release()
cv2.destroyAllWindows()                                                        #Destroying all the windows and releasing the cv2 objects created.